In [2]:
from instance_runner import InstanceRunner
from tqdm import tqdm
from validator import Validator

import pandas as pd
import json
import os

In [ ]:
runs = 1
output_folder = "/home/pedrowsl/cbm/output/10_09/"
comparison_file = "/home/pedrowsl/cbm/comparison.csv"
output_file = f"{output_folder}results_10_09.csv"
instances = ["a1"]
#instances = ["scpa1", "scpa2", "scpa3", "scpa4", "scpa5"]
parameters = {
    "threads": 15,
    "tempL": 15,
    "lkhS": 5,
    "tempMin": 0.01,
    "tempMax": 2.25,
    "constructionBias": 5.0,
    "MKL": 800,
    "PTL": 5000,
    "tempD": 3,
    "upType": 1,
    "tempUpdate": 7,
    "movementType": 2,
    "selectionBias": 1,
    "lkhMaxTime": 120
}

In [4]:
def get_comparison(df: pd.DataFrame, key: str, instance: str):
    return df.loc[df["Instance"] == instance, key].iloc[0]

In [ ]:
comparison = pd.read_csv("comparison.csv")
comparison = comparison.fillna(0)
comparison[["Exact", "Haddadi", "Soares"]] = comparison[["Exact", "Haddadi", "Soares"]].astype(int)

file_exists = os.path.isfile(output_file)

for instance in tqdm(instances):
    v = Validator(f"/home/pedrowsl/cbm/instances/{instance}")
    for _ in range(runs):
        ir = InstanceRunner(
            parameters=parameters,
            root_path="/home/pedrowsl/cbm",
            instance_path="/home/pedrowsl/cbm/instances/" + instance,
            header_path="/home/pedrowsl/cbm/src/CBMProblem.hpp",
        )
        output, elapsed = ir.run()
        output = json.loads(output)

        valid = v.validate(output['final_solution']['solution'], output['final_solution']['cost'])
        if not valid:
            raise Exception(f"Invalid S {output['final_solution']['solution']} and cost {output['final_solution']['cost']} for instance {instance}")

        df_row = pd.DataFrame([{
            'instance': instance,
            'exact': get_comparison(comparison, "Exact", instance),
            'pt': output['final_solution']['cost'],
            'lkh': min(x['cost'] for x in output['initial_solutions'] if x['construction'] == 'LKH'),
            'haddadi': get_comparison(comparison, "Haddadi", instance),
            'soares': get_comparison(comparison, "Soares", instance),
            'solution': output['final_solution']['solution'],
            'valid': valid,
            't(s)': elapsed
        }])

        with open(f"{output_folder}{instance}.json", "w") as f:
            f.write(json.dumps(output, indent=4))
        df_row.to_csv(output_file, mode='a', header=not file_exists, index=False)
        file_exists = True

  0%|          | 0/45 [00:00<?, ?it/s]

g++ -fopenmp -std=c++17 -O3 -march=native -c src/CBMSol.cpp -o src/CBMSol.o.prd
g++ -fopenmp -std=c++17 -O3 -march=native -c src/CBMProblem.cpp -o src/CBMProblem.o.prd


src/CBMProblem.cpp: In lambda function:
src/CBMProblem.cpp:152:5: warning: control reaches end of non-void function [-Wreturn-type]
  152 |     };
      |     ^


g++ -fopenmp -std=c++17 -O3 -march=native -c src/main.cpp -o src/main.o.prd
g++ -fopenmp -std=c++17 -O3 -march=native -o main_prd src/CBMSol.o.prd src/CBMProblem.o.prd src/main.o.prd
Compilation finished successfully.


 22%|██▏       | 10/45 [00:45<02:13,  3.81s/it]